In [ ]:
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.backends.cudnn as cudnn

In [ ]:
# 2 million word vectors trained on Common Crawl (600B tokens)
embedding_file = open('processed_data/crawl-300d-2M.vec')

In [ ]:
# The first line of the file contains the number of words in the vocabulary and the size of the vectors. 
# Each line contains a word followed by its vectors, like in the default fastText text format. 
# Each value is space separated.
# Words are ordered by descending frequency.
meta = embedding_file.readline()
num_words = int(meta.split()[0])
num_dims = int(meta.split()[1])

In [ ]:
# Some 'words' are not picked up by split(). Ignoring them.
# dictionary -> word : index
# reverse_dictionary -> index : word
dictionary = dict()
# To store embeddings for each word
embeddings = np.zeros((num_words, num_dims))
i = 0

# start of sequence token
dictionary['<pad>'] = len(dictionary)
embeddings[i] = np.random.rand(num_dims)
i += 1

# start of sequence token
dictionary['<sos>'] = len(dictionary)
embeddings[i] = np.random.rand(num_dims)
i += 1

# end of sequence token
dictionary['<eos>'] = len(dictionary)
embeddings[i] = np.random.rand(num_dims)
i += 1

# unkown word token
dictionary['<unk>'] = len(dictionary)
embeddings[i] = np.random.rand(num_dims)
i += 1

# Skip words not separated by split()
for line in embedding_file:
    items = line.split()
    if len(items) == (num_dims + 1):
        dictionary[items[0]] = len(dictionary)
        embeddings[i] = np.asarray([float(value) for value in items[1:]], dtype=np.float64)
        i += 1
reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
max_vocab = len(dictionary)
embeddings = embeddings[:max_vocab]
embedding_file.close()

print("Total words: %d" % max_vocab)
print("Number of dimensions: %d" % num_dims)

In [ ]:
input_vocab_size = 40000
output_vocab_size = 40000
# Input words and embeddings
input_dictionary = dict()
input_reverse_dictionary = dict()
input_embeddings = np.zeros((input_vocab_size, num_dims))
# Output words and embeddings
output_dictionary = dict()
output_reverse_dictionary = dict()
output_embeddings = np.zeros((output_vocab_size, num_dims))
# Extract input vocabulary
for i in range(input_vocab_size):
    word = reverse_dictionary[i]
    input_dictionary[word] = i
    input_reverse_dictionary[i] = word
    input_embeddings[i] = embeddings[i]
# Extract output vocabulary
for i in range(output_vocab_size):
    word = reverse_dictionary[i]
    output_dictionary[word] = i
    output_reverse_dictionary[i] = word
    output_embeddings = embeddings[i]
    
del dictionary, reverse_dictionary, embeddings
print('Input vocabulary size: %d' % input_vocab_size)
print('Output vocabulary size: %d' % output_vocab_size)

In [ ]:
# Load preprocessed titles and articles
titles = pickle.load(open('processed_data/titles', 'rb'))
articles = pickle.load(open('processed_data/articles', 'rb'))